In [99]:
import pandas

In [100]:
featuresDf = pandas.read_csv(filepath_or_buffer="features.csv")

In [101]:
featuresDf.describe()

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,MittelWX,VarianzX,StandardAX,MittelWY,VarianzY,StandardAY,...,MittelWPitch,VarianzPitch,StandardAPitch,MittelWRoll,VarianzRoll,StandardRoll,StartBewegungsD,StartBelichtung,Belichtungszeit,StopBewegungsD
count,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03
mean,1.519222e+12,53.614119,10.173735,17.370684,-0.769624,51.980899,3.425279,0.349991,21.042347,2.151565,...,-0.026104,0.014513,0.040029,-0.067708,0.037686,0.038152,2.963933e+13,2.963943e+13,1.150444e+07,2.963879e+13
std,1.756570e+05,0.001566,0.008892,3.760957,6.939281,106.924149,6.346005,5.175154,50.793803,4.052569,...,0.389855,0.074076,0.113304,0.540709,0.422125,0.190173,1.756579e+11,1.756574e+11,3.702001e+06,1.756334e+11
min,1.519222e+12,53.610455,10.156508,3.816000,-23.218000,0.000000,0.000000,-31.113000,0.000000,0.000000,...,-1.476000,0.000000,0.000000,-3.199000,0.000000,0.000000,2.932993e+13,2.932987e+13,8.963045e+06,2.932993e+13
25%,1.519222e+12,53.613506,10.166387,15.119999,-5.646750,0.000000,0.000000,-1.359500,0.000000,0.000000,...,-0.148250,0.000000,0.000000,-0.097000,0.000000,0.000000,2.948938e+13,2.949006e+13,1.000184e+07,2.948911e+13
50%,1.519223e+12,53.614555,10.176251,17.784000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.964318e+13,2.964389e+13,1.000184e+07,2.964258e+13
75%,1.519223e+12,53.615370,10.181249,20.052000,1.153750,4.621000,2.150000,2.342250,4.861750,2.205500,...,0.121250,0.000000,0.000000,0.011000,0.000000,0.000000,2.979034e+13,2.979062e+13,1.000184e+07,2.978940e+13
max,1.519223e+12,53.615856,10.185248,25.272000,27.957000,521.676000,22.841000,36.337000,442.166000,21.028000,...,1.579000,1.254000,1.120000,3.040000,8.715000,2.953000,2.993980e+13,2.993975e+13,2.999632e+07,2.993980e+13


In [79]:
featuresDf.head(5)

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,MittelWX,VarianzX,StandardAX,MittelWY,VarianzY,StandardAY,...,MittelWPitch,VarianzPitch,StandardAPitch,MittelWRoll,VarianzRoll,StandardRoll,StartBewegungsD,StartBelichtung,Belichtungszeit,StopBewegungsD
0,1519222189555,53.612377,10.176757,22.248,2.645,2.908,1.706,-7.172,206.2,14.36,...,-0.135,0.0,0.0,-0.349,0.0,0.0,29329927950266,29329873707614,10001839,29329931199954
1,1519222189850,53.612377,10.176757,22.248,0.000,0.000,0.000,0.000,0.0,0.00,...,0.000,0.0,0.0,0.000,0.0,0.0,29329927950266,29330174382614,10001839,29329931199954
2,1519222190122,53.612377,10.176757,22.248,-4.997,0.000,0.000,5.940,0.0,0.00,...,0.211,0.0,0.0,0.198,0.0,0.0,29330492257402,29330431604775,20003678,29329931199954
3,1519222190389,53.612324,10.176797,19.512,0.000,0.000,0.000,0.000,0.0,0.00,...,0.000,0.0,0.0,0.000,0.0,0.0,29330492257402,29330698871775,20003678,29329931199954
4,1519222190685,53.612324,10.176797,19.512,-4.629,0.000,0.000,11.115,0.0,0.00,...,-0.148,0.0,0.0,0.206,0.0,0.0,29331061552297,29330999509775,20003678,29329931199954


In [80]:
featuresDf.index

RangeIndex(start=0, stop=2000, step=1)

In [94]:
startSensorDataCollectionDf = featuresDf.StartBewegungsD
startExposureDf = featuresDf.StartBelichtung

Die Funktion berechnet den Mittelwert der Differenz zwischen zwei Dataframes mit einer Spalte.   
Prec.: startDf und endDf haben jeweils eine Spalte. Anzahl der Reihen der Dataframes > 0
Postc: Gibt den Mittelwert zurück   

In [111]:
def calcMeanTime(startDf,endDf):
    meanStartM = startDf.iloc[:].mean()
    meanStartE = endDf.iloc[:].mean()
    return meanStartM - meanStartE

In [112]:
calcMeanTime(startExposureDf,startSensorDataCollectionDf)

97487908.54296875

Berechnitt die maximale Differenz zwischen den Achsen von zwei Dataframes
Prec:
Postc.: Maximaler Wert

In [85]:
def calcMaxGap(startDf,endDf):
    startDf = startDf - endDf
    return startDf.max()

In [86]:
calcMaxGap(startExposureDf, startSensorDataCollectionDf,)

3156587736

In [105]:
startExposurePlusExposureTime = startExposureDf + featuresDf.Belichtungszeit
startExposurePlusExposureTime.head(2)

0    29329883709453
1    29330184384453
dtype: int64

In [106]:
featuresDf.Belichtungszeit.mean()

11504440.644

In [107]:
featuresDf.Belichtungszeit.max()

29996324

In [108]:
startExposurePlusExposureTime.max()

29939758811453

In [113]:
calcMeanTime(startExposurePlusExposureTime,featuresDf.StopBewegungsD)

647227348.0039062

In [114]:
calcMaxGap(startExposurePlusExposureTime, featuresDf.StopBewegungsD,)

6440767988